This Python Jupyter Notebook creates the SQL Server tables for the NFL model.

In [1]:
#Import required packages
import pyodbc
import sqlalchemy
from sqlalchemy import create_engine
import urllib

In [2]:
#Connect to database 'NFL' using SQlAlchemy
connection_str = "DRIVER={SQL SERVER};SERVER=LAPTOP-BJU2VQE0\SIMPLESERVER;DATABASE=NFL;TRUSTED_CONNECTION=YES"
params = urllib.parse.quote_plus(connection_str)
engine = create_engine('mssql+pyodbc:///?odbc_connect=%s' % params)
conn = engine.connect()

In [3]:
CreateTeams = """CREATE TABLE Teams
    (
    TeamID CHAR(3) NOT NULL,
    ShortName VARCHAR(3) PRIMARY KEY NOT NULL,
    FullName VARCHAR(25) NOT NULL,
    Nickname VARCHAR(15),
    Conference CHAR(3),
    Division VARCHAR(9),
    FoundingYear SMALLINT,
    Colour VARCHAR(15),
    )"""

In [4]:
engine.execute(CreateTeams)

In [5]:
CreateElo = """CREATE TABLE Elo
    (
    EloID AS CONCAT('E', EloYear, EloWeek, Team) PERSISTED PRIMARY KEY,
    YearWeek AS CONCAT(EloYear, '-', EloWeek) PERSISTED NOT NULL,
    EloYear SMALLINT NOT NULL,
    EloWeek TINYINT NOT NULL,
    Team VARCHAR(3) FOREIGN KEY REFERENCES Teams(ShortName) NOT NULL,
    Elo DECIMAL(10,5) NOT NULL,
    )"""

In [6]:
engine.execute(CreateElo)

In [7]:
CreateGames = """CREATE TABLE Games
    (
    GameID AS CONCAT ('G', GameYear, GameWeek, AwayTeam, HomeTeam) PERSISTED PRIMARY KEY,
    YearWeek AS CONCAT(GameYear,'-', GameWeek) PERSISTED NOT NULL,
    GameYear SMALLINT NOT NULL,
    GameWeek TINYINT NOT NULL,
    AwayTeam VARCHAR(3) FOREIGN KEY REFERENCES Teams(ShortName) NOT NULL,
    HomeTeam  VARCHAR(3) FOREIGN KEY REFERENCES Teams(ShortName) NOT NULL,
    NeutralVenue CHAR(1) NOT NULL,
    AwayScore TINYINT,
    HomeScore TINYINT,
    Winner AS CASE
        WHEN AwayScore IS NULL OR HomeScore IS NULL THEN 'INC'
        WHEN AwayScore > HomeScore THEN AwayTeam
        WHEN AwayScore < HomeScore THEN HomeTeam
        WHEN AwayScore = HomeScore THEN 'TIE'
        ELSE 'ERR' END
    PERSISTED,
    )"""

In [8]:
engine.execute(CreateGames)

In [9]:
CreateOdds = """CREATE TABLE Odds
    (
    OddsID AS CONCAT('O',GameID,OddsDateTime) PERSISTED PRIMARY KEY,
    GameID VARCHAR(17) FOREIGN KEY REFERENCES Games(GameID) NOT NULL,
    OddsDateTime DATETIME NOT NULL,
    AwayOddsFrac DECIMAL(10,5) NOT NULL,
    HomeOddsFrac DECIMAL(10,5) NOT NULL,
    )"""

In [10]:
engine.execute(CreateOdds)

In [11]:
CreateCalcs = """CREATE TABLE Calculations
    (
    CalcID AS CONCAT ('C', OddsID,CalcDateTime,AlgorithmRef) PERSISTED PRIMARY KEY,
    OddsID VARCHAR(58) FOREIGN KEY REFERENCES Odds(OddsID) NOT NULL,
    CalcDateTime DATETIME NOT NULL,
    BetTeam VARCHAR(3) FOREIGN KEY REFERENCES Teams(ShortName),
    BetOddsFrac DECIMAL(10,5) NOT NULL,
    BetUnits DECIMAL(4,2) NOT NULL,
    WinUnits DECIMAL(4,2) NOT NULL,
    AlgorithmRef VARCHAR(100) NOT NULL,
    AwayEloFrac DECIMAL(10,5) NOT NULL,
    AwayEloPC DECIMAL(7,5) NOT NULL,
    AwayOddsFrac DECIMAL(10,5) NOT NULL,
    AwayOddsPC DECIMAL(7,5) NOT NULL,
    AwayEdgeFrac DECIMAL(10,5) NOT NULL,
    AwayEdgePC DECIMAL(7,5) NOT NULL,
    HomeEloFrac DECIMAL(10,5) NOT NULL,
    HomeEloPC DECIMAL(7,5) NOT NULL,
    HomeOddsFrac DECIMAL(10,5) NOT NULL,
    HomeOddsPC DECIMAL(7,5) NOT NULL,
    HomeEdgeFrac DECIMAL(10,5) NOT NULL,
    HomeEdgePC DECIMAL(7,5) NOT NULL,
    KellyPC DECIMAL(7,5) NOT NULL,
    )"""

In [12]:
engine.execute(CreateCalcs)

In [13]:
CreateBets = """Create Table Bets
    (
    BetID AS CONCAT('B',CalcID,BetDateTime) PERSISTED PRIMARY KEY,
    CalcID VARCHAR(199) FOREIGN KEY REFERENCES Calculations(CalcID) NOT NULL,
    BetDateTime DATETIME NOT NULL,
    BetTeam VARCHAR(3) FOREIGN KEY REFERENCES Teams(ShortName) NOT NULL,
    BetAmount DECIMAL(10,2) NOT NULL,
    WinningsAmount DECIMAL(10,2) NOT NULL,
    Profit DECIMAL(10,2),
    )"""

In [14]:
engine.execute(CreateBets)

In [15]:
#Get the table names currently in the database
insp = sqlalchemy.inspect(engine)
print(insp.get_table_names())

['Bets', 'Calculations', 'Elo', 'Games', 'Odds', 'Teams']


In [16]:
conn.close()